# Analiza rynku nieruchomości w Polsce

In [1]:
!pip3 install pandas numpy matplotlib seaborn plotly scikit-learn openpyxl
!pip3 install "statsmodels==0.14.4" "scipy==1.15.3" yfinance fredapi


[notice] A new release of pip is available: 23.1.2 -> 25.1.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.1.2 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


In [2]:
# Komórka 2: Import bibliotek i konfiguracja
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import warnings
warnings.filterwarnings('ignore')

# Biblioteki do analizy statystycznej
from scipy import stats
from scipy.stats import pearsonr, spearmanr, normaltest
import statsmodels.api as sm
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.arima.model import ARIMA

# Biblioteki machine learning
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error

# Konfiguracja wyświetlania
plt.rcParams['figure.figsize'] = (12, 8)
plt.style.use('seaborn-v0_8')
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

print("✅ Wszystkie biblioteki zostały załadowane pomyślnie!")


✅ Wszystkie biblioteki zostały załadowane pomyślnie!


In [ ]:
import pandas as pd

# Load Excel file
file_path = "ceny_mieszkan.xlsx"
sheet_primary = pd.read_excel(file_path, sheet_name="Rynek pierwotny", header=None)
sheet_secondary = pd.read_excel(file_path, sheet_name="Rynek wtórny", header=None)

# Find the starting row (header) for the data — the row containing 'Kwartał'
def find_data_start(df):
    return df[df.apply(lambda row: row.astype(str).str.contains("Kwartał", case=False).any(), axis=1)].index[0]


# Extract offer and transaction prices from specified columns
def split_offer_trans(df, start_row, offer_cols=18, skip_cols=5, trans_cols=18):
    df_data = df.iloc[start_row:82]
    df_data.columns = df_data.iloc[0]  # set headers
    df_data = df_data[1:].reset_index(drop=True)

    offer_df = df_data.iloc[:, :offer_cols]
    trans_df = df_data.iloc[:, offer_cols + skip_cols : offer_cols + skip_cols + trans_cols]
    return offer_df, trans_df

# Apply to both sheets
start_primary = find_data_start(sheet_primary)
primary_offer, primary_trans = split_offer_trans(sheet_primary, start_primary)

start_secondary = find_data_start(sheet_secondary)
secondary_offer, secondary_trans = split_offer_trans(sheet_secondary, start_secondary)

miasta = primary_offer.columns[1:].tolist()

['Białystok', 'Bydgoszcz', 'Gdańsk', 'Gdynia*', 'Katowice', 'Kielce', 'Kraków', 'Lublin', 'Łódź', 'Olsztyn', 'Opole', 'Poznań', 'Rzeszów', 'Szczecin', 'Warszawa', 'Wrocław', 'Zielona Góra']


In [60]:
# odczyt danych o inflacji
# Load Excel file
inflacja_path = "bazowa.xlsx"
sheet_inflacja = pd.read_excel(inflacja_path, sheet_name="dane_kwartalne", header=None)

df_inflacja = sheet_inflacja.iloc[25:100][[0, 6]]
# Rename columns for clarity
df_inflacja.columns = ['Kwartał', 'CPI']

print(df_inflacja.head())

         Kwartał    CPI
25  III kw. 2006  100.2
26   IV kw. 2006  100.3
27    I kw. 2007  100.7
28   II kw. 2007  101.2
29  III kw. 2007   99.9


## TODO

In [ ]:
# Komórka 4: Czyszczenie i feature engineering danych
def przemiana_daty_kwartaly(df):
    """
    Konwertuje kwartały na daty i dodaje zmienne czasowe
    """
    print("📅 Przetwarzanie dat i tworzenie zmiennych czasowych...")
    
    def kwartal_do_daty(kwartal_str):
        """Konwertuje string kwartału na datę"""
        try:
            # Rozdziel kwartał i rok
            czesci = kwartal_str.strip().split()
            if len(czesci) == 3:
                czesci = [czesci[0], czesci[-1]]
            elif len(czesci) != 2:
                return None
                
            kwartal_rom, rok = czesci
            
            # Mapowanie kwartałów rzymskich na miesiące
            kwartaly_map = {
                'I': '01-01',
                'II': '04-01', 
                'III': '07-01',
                'IV': '10-01'
            }
            
            if kwartal_rom in kwartaly_map:
                data_str = f"{rok}-{kwartaly_map[kwartal_rom]}"
                return pd.to_datetime(data_str)
            else:
                return None
                
        except Exception as e:
            print(f"Błąd przy konwersji {kwartal_str}: {e}")
            return None
    
    # Konwersja kwartałów na daty
    df['data'] = df['Kwartał'].apply(kwartal_do_daty)
    
    # Usuń rekordy z błędnymi datami
    df = df.dropna(subset=['data'])
    
    # Dodaj zmienne czasowe
    df['rok'] = df['data'].dt.year
    df['kwartal_num'] = df['data'].dt.quarter
    df['indeks_czasowy'] = range(len(df))
    
    # Sortowanie według daty
    df = df.sort_values('data').reset_index(drop=True)
    
    return df

def feature_engineering(df):
    """
    Tworzy dodatkowe cechy dla analizy
    """
    print("🔧 Tworzenie cech analitycznych...")
    
    # Zmienne lagowane
    for miasto in miasta:
        for rynek in ['pierwotny', 'wtórny']:
            mask = (df['miasto'] == miasto) & (df['rynek'] == rynek)
            df.loc[mask, 'cena_lag1'] = df.loc[mask, 'cena_m2'].shift(1)
            df.loc[mask, 'cena_lag2'] = df.loc[mask, 'cena_m2'].shift(2)
            df.loc[mask, 'cena_lag4'] = df.loc[mask, 'cena_m2'].shift(4)
    
    # Zmiany rok do roku (YoY)
    for miasto in miasta:
        for rynek in ['pierwotny', 'wtórny']:
            mask = (df['miasto'] == miasto) & (df['rynek'] == rynek)
            df.loc[mask, 'cena_yoy'] = df.loc[mask, 'cena_m2'].pct_change(periods=4) * 100
    
    # Średnie kroczące
    for miasto in miasta:
        for rynek in ['pierwotny', 'wtórny']:
            mask = (df['miasto'] == miasto) & (df['rynek'] == rynek)
            df.loc[mask, 'cena_ma4'] = df.loc[mask, 'cena_m2'].rolling(window=4).mean()
            df.loc[mask, 'cena_ma8'] = df.loc[mask, 'cena_m2'].rolling(window=8).mean()
    
    # Ceny realne (skorygowane o inflację)
    df['cena_realna'] = df['cena_m2'] / (1 + df['inflacja_cpi']/100)
    
    # Trendy kwadratowe
    df['rok_kwadrat'] = df['rok'] ** 2
    df['indeks_kwadrat'] = df['indeks_czasowy'] ** 2
    
    return df

def sanitizacja_danych(df):
    """
    Czyści dane i obsługuje wartości odstające
    """
    print("🧹 Sanityzacja danych...")
    
    # Usuń rekordy z brakującymi cenami
    df = df.dropna(subset=['cena_m2'])
    
    # Detekcja outlierów metodą IQR
    Q1 = df['cena_m2'].quantile(0.25)
    Q3 = df['cena_m2'].quantile(0.75)
    IQR = Q3 - Q1
    
    # Określ granice outlierów
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    
    # Zlicz outliery
    outliers = df[(df['cena_m2'] < lower_bound) | (df['cena_m2'] > upper_bound)]
    print(f"🔍 Znaleziono {len(outliers)} outlierów ({len(outliers)/len(df)*100:.1f}%)")
    
    # Zastąp outliery wartościami granicznymi (winsoryzacja)
    df.loc[df['cena_m2'] < lower_bound, 'cena_m2'] = lower_bound
    df.loc[df['cena_m2'] > upper_bound, 'cena_m2'] = upper_bound
    
    return df

df_kwart = przemiana_daty_kwartaly(primary_offer)
print(df_kwart.head())

# # Zastosowanie funkcji przetwarzania
if 'df_mieszkania' in locals():
    df_mieszkania = przemiana_daty_kwartaly(df_mieszkania)
    df_mieszkania = sanitizacja_danych(df_mieszkania)
    df_mieszkania = feature_engineering(df_mieszkania)
    
    print("✅ Przetwarzanie danych zakończone!")
    print(f"📊 Finalne dane: {df_mieszkania.shape}")
    print("\n📋 Kolumny w zbiorze danych:")
    print(df_mieszkania.columns.tolist())
    
#     # Wyświetl statystyki opisowe
    print("\n📈 Statystyki opisowe cen:")
    print(df_mieszkania.groupby(['miasto', 'rynek'])['cena_m2'].describe())


📅 Przetwarzanie dat i tworzenie zmiennych czasowych...
6   Kwartał Białystok Bydgoszcz Gdańsk Gdynia* Katowice       Kielce Kraków  \
0  III 2006      2727      3243   5004    4937  3986.27  2918.644443   6021   
1   IV 2006      2727      3947   6199    5284  4062.31  3492.617589   7929   
2    I 2007      4181      4501   6437    6466  4518.37  3381.718956   8247   
3   II 2007      4725      5700   7999    8934  5180.56   3901.57831   8129   
4  III 2007      4199      4994   8053    8114  6235.53  4651.056569   8420   

6 Lublin  Łódź Olsztyn        Opole       Poznań Rzeszów     Szczecin  \
0   3456  3210    4000  3342.843092  4769.933415    2862  3214.980636   
1   3314  4237    4800          NaN    4827.2275    3401  3679.123336   
2   3604  4927    5149          NaN  7274.746093    3420  4749.124321   
3   4143  5453    4776          NaN  7360.018661    4080  5049.969372   
4   4645  5201    4900  4784.143691   6837.76137    4069  5034.720763   

6 Warszawa      Wrocław Zielona

## TODO

In [ ]:
# Komórka 5: Analiza statystyczna
def analiza_korelacji(df):
    """
    Przeprowadza analizę korelacji między zmiennymi
    """
    print("🔗 Analiza korelacji...")
    
    # Wybierz zmienne numeryczne
    zmienne_numeryczne = ['cena_m2', 'inflacja_cpi', 'wibor_3m', 'rok', 'cena_realna']
    
    # Macierz korelacji
    korelacje = df[zmienne_numeryczne].corr()
    
    # Wizualizacja macierzy korelacji
    fig, ax = plt.subplots(figsize=(10, 8))
    sns.heatmap(korelacje, annot=True, cmap='coolwarm', center=0, 
                square=True, fmt='.3f', cbar_kws={'shrink': 0.8})
    plt.title('Macierz Korelacji - Zmienne Ekonomiczne vs Ceny Mieszkań')
    plt.tight_layout()
    plt.show()
    
    # Szczegółowa analiza korelacji
    rezultaty_korelacji = {}
    
    for miasto in ['Warszawa', 'Kraków']:
        for rynek in ['pierwotny', 'wtórny']:
            mask = (df['miasto'] == miasto) & (df['rynek'] == rynek)
            dane_segmentu = df[mask].dropna()
            
            if len(dane_segmentu) > 10:
                # Korelacja z inflacją
                corr_inflacja, p_inflacja = pearsonr(dane_segmentu['cena_m2'], dane_segmentu['inflacja_cpi'])
                
                # Korelacja z WIBOR
                corr_wibor, p_wibor = pearsonr(dane_segmentu['cena_m2'], dane_segmentu['wibor_3m'])
                
                rezultaty_korelacji[f"{miasto}_{rynek}"] = {
                    'inflacja_corr': corr_inflacja,
                    'inflacja_p': p_inflacja,
                    'wibor_corr': corr_wibor,
                    'wibor_p': p_wibor
                }
    
    # Wyświetl rezultaty
    print("\n📊 Korelacje Pearsona (ceny mieszkań vs wskaźniki ekonomiczne):")
    for segment, stats in rezultaty_korelacji.items():
        print(f"\n{segment}:")
        print(f"  Inflacja CPI: r={stats['inflacja_corr']:.3f}, p={stats['inflacja_p']:.3f}")
        print(f"  WIBOR 3M:     r={stats['wibor_corr']:.3f}, p={stats['wibor_p']:.3f}")
    
    return rezultaty_korelacji

def testy_statystyczne(df):
    """
    Przeprowadza testy statystyczne
    """
    print("🧪 Testy statystyczne...")
    
    rezultaty_testow = {}
    
    for miasto in ['Warszawa', 'Kraków']:
        for rynek in ['pierwotny', 'wtórny']:
            mask = (df['miasto'] == miasto) & (df['rynek'] == rynek)
            ceny = df[mask]['cena_m2'].dropna()
            
            if len(ceny) > 8:
                # Test normalności Shapiro-Wilka
                stat_shapiro, p_shapiro = stats.shapiro(ceny)
                
                # Test trendu (regresja liniowa względem czasu)
                indeksy = df[mask]['indeks_czasowy'].values
                if len(indeksy) == len(ceny):
                    slope, intercept, r_value, p_trend, std_err = stats.linregress(indeksy, ceny)
                else:
                    slope, p_trend, r_value = np.nan, np.nan, np.nan
                
                rezultaty_testow[f"{miasto}_{rynek}"] = {
                    'shapiro_stat': stat_shapiro,
                    'shapiro_p': p_shapiro,
                    'trend_slope': slope,
                    'trend_p': p_trend,
                    'trend_r2': r_value**2 if not np.isnan(r_value) else np.nan
                }
    
    # Wyświetl rezultaty testów
    print("\n📋 Rezultaty testów statystycznych:")
    for segment, stats in rezultaty_testow.items():
        print(f"\n{segment}:")
        print(f"  Test normalności (Shapiro-Wilk): stat={stats['shapiro_stat']:.4f}, p={stats['shapiro_p']:.4f}")
        if stats['shapiro_p'] < 0.05:
            print("    ❌ Rozkład NIE jest normalny")
        else:
            print("    ✅ Rozkład jest normalny")
            
        print(f"  Trend czasowy: slope={stats['trend_slope']:.2f} PLN/m²/kw, p={stats['trend_p']:.4f}, R²={stats['trend_r2']:.3f}")
        if stats['trend_p'] < 0.05:
            print("    📈 Trend jest statystycznie istotny")
        else:
            print("    ➡️ Brak istotnego trendu")
    
    return rezultaty_testow

# Przeprowadzenie analiz statystycznych
if 'df_mieszkania' in locals():
    korelacje = analiza_korelacji(df_mieszkania)
    testy = testy_statystyczne(df_mieszkania)


## TODO

In [ ]:
# Komórka 6: Interaktywne wizualizacje
def utworz_wykres_czasowy_interaktywny(df):
    """
    Tworzy interaktywny wykres czasowy cen mieszkań
    """
    print("📊 Tworzenie interaktywnego wykresu czasowego...")
    
    fig = make_subplots(
        rows=3, cols=1,
        subplot_titles=(
            'Ceny Mieszkań w Warszawie i Krakowie',
            'Wskaźniki Ekonomiczne',
            'Zmiany Rok do Roku (%)'
        ),
        vertical_spacing=0.08,
        specs=[[{"secondary_y": False}],
               [{"secondary_y": True}],
               [{"secondary_y": False}]]
    )
    
    # Kolory dla różnych segmentów
    kolory = {
        'Warszawa_pierwotny': '#1f77b4',
        'Warszawa_wtórny': '#ff7f0e', 
        'Kraków_pierwotny': '#2ca02c',
        'Kraków_wtórny': '#d62728'
    }
    
    # Dodaj linie cen mieszkań
    for miasto in ['Warszawa', 'Kraków']:
        for rynek in ['pierwotny', 'wtórny']:
            mask = (df['miasto'] == miasto) & (df['rynek'] == rynek)
            dane_segmentu = df[mask].sort_values('data')
            
            if len(dane_segmentu) > 0:
                fig.add_trace(
                    go.Scatter(
                        x=dane_segmentu['data'],
                        y=dane_segmentu['cena_m2'],
                        mode='lines+markers',
                        name=f'{miasto} - {rynek}',
                        line=dict(color=kolory[f'{miasto}_{rynek}'], width=2),
                        hovertemplate='<b>%{fullData.name}</b><br>' +
                                    'Data: %{x}<br>' +
                                    'Cena: %{y:,.0f} PLN/m²<extra></extra>'
                    ),
                    row=1, col=1
                )
    
    # Dodaj wskaźniki ekonomiczne do drugiego wykresu
    dane_ekonomiczne = df.drop_duplicates('data').sort_values('data')
    
    fig.add_trace(
        go.Scatter(
            x=dane_ekonomiczne['data'],
            y=dane_ekonomiczne['inflacja_cpi'],
            mode='lines+markers',
            name='Inflacja CPI (%)',
            line=dict(color='red', width=2),
            yaxis='y3'
        ),
        row=2, col=1
    )
    
    fig.add_trace(
        go.Scatter(
            x=dane_ekonomiczne['data'],
            y=dane_ekonomiczne['wibor_3m'],
            mode='lines+markers', 
            name='WIBOR 3M (%)',
            line=dict(color='blue', width=2),
            yaxis='y4'
        ),
        row=2, col=1
    )
    
    # Dodaj zmiany YoY do trzeciego wykresu
    for miasto in ['Warszawa', 'Kraków']:
        for rynek in ['pierwotny', 'wtórny']:
            mask = (df['miasto'] == miasto) & (df['rynek'] == rynek)
            dane_segmentu = df[mask].dropna(subset=['cena_yoy']).sort_values('data')
            
            if len(dane_segmentu) > 0:
                fig.add_trace(
                    go.Scatter(
                        x=dane_segmentu['data'],
                        y=dane_segmentu['cena_yoy'],
                        mode='lines+markers',
                        name=f'{miasto} - {rynek} YoY',
                        line=dict(color=kolory[f'{miasto}_{rynek}'], width=2, dash='dot'),
                        showlegend=False
                    ),
                    row=3, col=1
                )
    
    # Aktualizacja layoutu
    fig.update_layout(
        height=900,
        title_text="Analiza Polskiego Rynku Nieruchomości 2006-2025",
        title_x=0.5,
        showlegend=True,
        hovermode='x unified'
    )
    
    # Aktualizacja osi Y
    fig.update_yaxes(title_text="Cena PLN/m²", row=1, col=1)
    fig.update_yaxes(title_text="Inflacja (%)", row=2, col=1)
    fig.update_yaxes(title_text="WIBOR (%)", secondary_y=True, row=2, col=1)
    fig.update_yaxes(title_text="Zmiana YoY (%)", row=3, col=1)
    
    # Aktualizacja osi X
    fig.update_xaxes(title_text="Rok", row=3, col=1)
    
    return fig

def utworz_dashboard_analityczny(df):
    """
    Tworzy dashboard z wieloma wykresami analitycznymi
    """
    print("📊 Tworzenie dashboardu analitycznego...")
    
    fig = make_subplots(
        rows=2, cols=2,
        subplot_titles=(
            'Porównanie Cen: Warszawa vs Kraków',
            'Rozkład Cen Mieszkań',
            'Ceny vs Inflacja CPI',
            'Ceny vs WIBOR 3M'
        ),
        specs=[[{"type": "scatter"}, {"type": "histogram"}],
               [{"type": "scatter"}, {"type": "scatter"}]]
    )
    
    # 1. Porównanie Warszawa vs Kraków
    for rynek in ['pierwotny', 'wtórny']:
        warszawa = df[(df['miasto'] == 'Warszawa') & (df['rynek'] == rynek)]
        krakow = df[(df['miasto'] == 'Kraków') & (df['rynek'] == rynek)]
        
        if len(warszawa) > 0 and len(krakow) > 0:
            fig.add_trace(
                go.Scatter(
                    x=warszawa['cena_m2'],
                    y=krakow['cena_m2'],
                    mode='markers',
                    name=f'Rynek {rynek}',
                    marker=dict(size=8, opacity=0.7)
                ),
                row=1, col=1
            )
    
    # Dodaj linię y=x dla porównania
    min_cena = df['cena_m2'].min()
    max_cena = df['cena_m2'].max()
    fig.add_trace(
        go.Scatter(
            x=[min_cena, max_cena],
            y=[min_cena, max_cena],
            mode='lines',
            name='y = x',
            line=dict(dash='dash', color='gray'),
            showlegend=False
        ),
        row=1, col=1
    )
    
    # 2. Histogram rozkładu cen
    for miasto in ['Warszawa', 'Kraków']:
        dane_miasta = df[df['miasto'] == miasto]['cena_m2']
        fig.add_trace(
            go.Histogram(
                x=dane_miasta,
                name=miasto,
                opacity=0.7,
                nbinsx=30
            ),
            row=1, col=2
        )
    
    # 3. Ceny vs Inflacja
    fig.add_trace(
        go.Scatter(
            x=df['inflacja_cpi'],
            y=df['cena_m2'],
            mode='markers',
            name='Ceny vs Inflacja',
            marker=dict(color=df['rok'], colorscale='Viridis', size=6),
            text=df['kwartal'],
            hovertemplate='Inflacja: %{x:.1f}%<br>Cena: %{y:,.0f} PLN/m²<br>%{text}<extra></extra>'
        ),
        row=2, col=1
    )
    
    # 4. Ceny vs WIBOR
    fig.add_trace(
        go.Scatter(
            x=df['wibor_3m'],
            y=df['cena_m2'],
            mode='markers',
            name='Ceny vs WIBOR',
            marker=dict(color=df['rok'], colorscale='Plasma', size=6),
            text=df['kwartal'],
            hovertemplate='WIBOR: %{x:.1f}%<br>Cena: %{y:,.0f} PLN/m²<br>%{text}<extra></extra>'
        ),
        row=2, col=2
    )
    
    # Aktualizacja layoutu
    fig.update_layout(
        height=800,
        title_text="Dashboard Analityczny - Polski Rynek Nieruchomości",
        title_x=0.5,
        showlegend=True
    )
    
    # Aktualizacja etykiet osi
    fig.update_xaxes(title_text="Cena Warszawa (PLN/m²)", row=1, col=1)
    fig.update_yaxes(title_text="Cena Kraków (PLN/m²)", row=1, col=1)
    
    fig.update_xaxes(title_text="Cena (PLN/m²)", row=1, col=2)
    fig.update_yaxes(title_text="Częstość", row=1, col=2)
    
    fig.update_xaxes(title_text="Inflacja CPI (%)", row=2, col=1)
    fig.update_yaxes(title_text="Cena (PLN/m²)", row=2, col=1)
    
    fig.update_xaxes(title_text="WIBOR 3M (%)", row=2, col=2)
    fig.update_yaxes(title_text="Cena (PLN/m²)", row=2, col=2)
    
    return fig

# Tworzenie wizualizacji
if 'df_mieszkania' in locals():
    wykres_czasowy = utworz_wykres_czasowy_interaktywny(df_mieszkania)
    wykres_czasowy.show()
    
    dashboard = utworz_dashboard_analityczny(df_mieszkania) 
    dashboard.show()


## TODO

In [ ]:
# Komórka 7: Modele uczenia maszynowego
def przygotuj_dane_do_modelowania(df):
    """
    Przygotowuje dane do modelowania ML
    """
    print("🤖 Przygotowywanie danych do modelowania...")
    
    # Skupiamy się na Warszawie dla uproszczenia
    dane_warszawa = df[df['miasto'] == 'Warszawa'].copy()
    
    # Usuń rekordy z brakującymi wartościami w kluczowych kolumnach
    dane_warszawa = dane_warszawa.dropna(subset=['cena_m2', 'inflacja_cpi', 'wibor_3m'])
    
    # Kodowanie zmiennych kategorycznych
    le_rynek = LabelEncoder()
    dane_warszawa['rynek_encoded'] = le_rynek.fit_transform(dane_warszawa['rynek'])
    
    # Wybór cech do modelowania
    cechy = ['rok', 'kwartal_num', 'indeks_czasowy', 'inflacja_cpi', 'wibor_3m', 
             'rynek_encoded', 'rok_kwadrat']
    
    # Dodaj cechy lagowane jeśli dostępne
    for col in ['cena_lag1', 'cena_lag2', 'cena_ma4']:
        if col in dane_warszawa.columns:
            cechy.append(col)
    
    # Usuń rekordy z NaN w cechach
    dane_kompletne = dane_warszawa.dropna(subset=cechy + ['cena_m2'])
    
    X = dane_kompletne[cechy]
    y = dane_kompletne['cena_m2']
    
    print(f"📊 Przygotowano {len(X)} próbek z {len(cechy)} cechami")
    print(f"📋 Cechy: {cechy}")
    
    return X, y, dane_kompletne, le_rynek

def trenuj_modele_ml(X, y):
    """
    Trenuje różne modele ML i porównuje ich wydajność
    """
    print("🏋️ Trenowanie modeli uczenia maszynowego...")
    
    # Podział danych (uwaga na kolejność czasową)
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=42, shuffle=False
    )
    
    # Skalowanie cech
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    
    # Słownik modeli
    modele = {
        'Linear Regression': LinearRegression(),
        'Random Forest': RandomForestRegressor(n_estimators=100, random_state=42),
        'Gradient Boosting': GradientBoostingRegressor(n_estimators=100, random_state=42)
    }
    
    # Trenowanie i ewaluacja modeli
    wyniki_modeli = {}
    
    for nazwa_modelu, model in modele.items():
        print(f"🔧 Trenowanie {nazwa_modelu}...")
        
        if nazwa_modelu == 'Linear Regression':
            # Użyj skalowanych danych dla regresji liniowej
            model.fit(X_train_scaled, y_train)
            y_pred = model.predict(X_test_scaled)
        else:
            # Użyj oryginalnych danych dla modeli drzewiastych
            model.fit(X_train, y_train)
            y_pred = model.predict(X_test)
        
        # Oblicz metryki
        mse = mean_squared_error(y_test, y_pred)
        rmse = np.sqrt(mse)
        mae = mean_absolute_error(y_test, y_pred)
        r2 = r2_score(y_test, y_pred)
        
        # Oblicz błąd względny (MAPE)
        mape = np.mean(np.abs((y_test - y_pred) / y_test)) * 100
        
        wyniki_modeli[nazwa_modelu] = {
            'model': model,
            'predictions': y_pred,
            'mse': mse,
            'rmse': rmse,
            'mae': mae,
            'r2': r2,
            'mape': mape
        }
        
        print(f"✅ {nazwa_modelu} - R²: {r2:.4f}, RMSE: {rmse:.0f}, MAE: {mae:.0f}, MAPE: {mape:.2f}%")
    
    return wyniki_modeli, X_test, y_test, scaler

def wizualizacja_porownania_modeli(wyniki_modeli, X_test, y_test):
    """
    Tworzy wizualizację porównania modeli
    """
    print("📊 Tworzenie wizualizacji porównania modeli...")
    
    fig = make_subplots(
        rows=2, cols=2,
        subplot_titles=(
            'Porównanie Predykcji vs Rzeczywiste',
            'Metryki Wydajności Modeli',
            'Błędy Predykcji',
            'Ważność Cech (Random Forest)'
        )
    )
    
    kolory_modeli = ['blue', 'green', 'red']
    
    # 1. Predykcje vs rzeczywiste wartości
    for i, (nazwa, wyniki) in enumerate(wyniki_modeli.items()):
        fig.add_trace(
            go.Scatter(
                x=y_test,
                y=wyniki['predictions'],
                mode='markers',
                name=nazwa,
                marker=dict(color=kolory_modeli[i], size=6, opacity=0.7)
            ),
            row=1, col=1
        )
    
    # Dodaj linię idealną
    min_val, max_val = y_test.min(), y_test.max()
    fig.add_trace(
        go.Scatter(
            x=[min_val, max_val],
            y=[min_val, max_val],
            mode='lines',
            name='Idealna predykcja',
            line=dict(dash='dash', color='gray'),
            showlegend=False
        ),
        row=1, col=1
    )
    
    # 2. Metryki wydajności
    metryki = ['R²', 'RMSE', 'MAE', 'MAPE']
    nazwy_modeli = list(wyniki_modeli.keys())
    
    for i, metryka in enumerate(['r2', 'rmse', 'mae', 'mape']):
        wartosci = [wyniki_modeli[model][metryka] for model in nazwy_modeli]
        fig.add_trace(
            go.Bar(
                x=nazwy_modeli,
                y=wartosci,
                name=metryki[i],
                showlegend=False
            ),
            row=1, col=2
        )
    
    # 3. Rozkład błędów
    for i, (nazwa, wyniki) in enumerate(wyniki_modeli.items()):
        bledy = y_test - wyniki['predictions']
        fig.add_trace(
            go.Box(
                y=bledy,
                name=nazwa,
                showlegend=False
            ),
            row=2, col=1
        )
    
    # 4. Ważność cech (tylko dla Random Forest)
    if 'Random Forest' in wyniki_modeli:
        rf_model = wyniki_modeli['Random Forest']['model']
        waznosci = rf_model.feature_importances_
        nazwy_cech = X_test.columns
        
        # Sortuj według ważności
        sorted_idx = np.argsort(waznosci)[::-1]
        
        fig.add_trace(
            go.Bar(
                x=waznosci[sorted_idx][:10],  # Top 10 cech
                y=[nazwy_cech[i] for i in sorted_idx[:10]],
                orientation='h',
                showlegend=False
            ),
            row=2, col=2
        )
    
    # Aktualizacja layoutu
    fig.update_layout(
        height=800,
        title_text="Porównanie Modeli Predykcyjnych - Ceny Mieszkań w Warszawie",
        title_x=0.5
    )
    
    # Aktualizacja etykiet osi
    fig.update_xaxes(title_text="Rzeczywiste ceny (PLN/m²)", row=1, col=1)
    fig.update_yaxes(title_text="Predykcje (PLN/m²)", row=1, col=1)
    
    fig.update_xaxes(title_text="Model", row=1, col=2)
    fig.update_yaxes(title_text="Wartość metryki", row=1, col=2)
    
    fig.update_xaxes(title_text="Model", row=2, col=1)
    fig.update_yaxes(title_text="Błąd predykcji (PLN/m²)", row=2, col=1)
    
    fig.update_xaxes(title_text="Ważność", row=2, col=2)
    fig.update_yaxes(title_text="Cecha", row=2, col=2)
    
    return fig

# Trenowanie modeli
if 'df_mieszkania' in locals():
    X, y, dane_kompletne, le_rynek = przygotuj_dane_do_modelowania(df_mieszkania)
    wyniki_modeli, X_test, y_test, scaler = trenuj_modele_ml(X, y)
    
    # Wizualizacja wyników
    wykres_modeli = wizualizacja_porownania_modeli(wyniki_modeli, X_test, y_test)
    wykres_modeli.show()
    
    # Podsumowanie wyników
    print("\n🏆 PODSUMOWANIE WYDAJNOŚCI MODELI:")
    print("="*60)
    for nazwa, wyniki in wyniki_modeli.items():
        print(f"{nazwa:20} | R²: {wyniki['r2']:6.4f} | RMSE: {wyniki['rmse']:7.0f} | MAE: {wyniki['mae']:6.0f} | MAPE: {wyniki['mape']:5.2f}%")


In [ ]:
# Komórka 8: Prognozy cenowe
def utworz_prognozy_cenowe(najlepszy_model, scaler, dane_kompletne, X, y):
    """
    Tworzy prognozy cen mieszkań na przyszłe okresy
    """
    print("🔮 Tworzenie prognoz cenowych...")
    
    # Znajdź najlepszy model na podstawie R²
    najlepsza_nazwa = max(wyniki_modeli.keys(), key=lambda k: wyniki_modeli[k]['r2'])
    model = wyniki_modeli[najlepsza_nazwa]['model']
    
    print(f"🏆 Używany model: {najlepsza_nazwa}")
    
    # Przygotuj dane do prognozowania (ostatnie obserwacje)
    ostatnie_dane = dane_kompletne.iloc[-1:].copy()
    
    # Prognozy na 8 kolejnych kwartałów (2 lata)
    prognozy = []
    daty_prognoz = []
    
    for i in range(8):
        # Przygotuj cechy dla prognozy
        ostatni_rok = int(ostatnie_dane['rok'].iloc[0])
        ostatni_kwartal = int(ostatnie_dane['kwartal_num'].iloc[0])
        ostatni_indeks = int(ostatnie_dane['indeks_czasowy'].iloc[0])
        
        # Oblicz nowe wartości czasowe
        nowy_kwartal = ostatni_kwartal + i + 1
        nowy_rok = ostatni_rok + (nowy_kwartal - 1) // 4
        nowy_kwartal = ((nowy_kwartal - 1) % 4) + 1
        nowy_indeks = ostatni_indeks + i + 1
        
        # Załóż przyszłe wartości ekonomiczne (można poprawić o prognozy makroekonomiczne)
        przyszla_inflacja = 3.5  # Założona inflacja docelowa NBP
        przyszly_wibor = 4.0     # Założona normalizacja stóp procentowych
        
        # Utwórz wektor cech
        cechy_prognozy = {
            'rok': nowy_rok,
            'kwartal_num': nowy_kwartal,
            'indeks_czasowy': nowy_indeks,
            'inflacja_cpi': przyszla_inflacja,
            'wibor_3m': przyszly_wibor,
            'rynek_encoded': 0,  # pierwotny
            'rok_kwadrat': nowy_rok ** 2
        }
        
        # Dodaj cechy lagowane (użyj ostatnich znanych wartości lub poprzednich prognoz)
        if i == 0:
            if 'cena_lag1' in X.columns:
                cechy_prognozy['cena_lag1'] = dane_kompletne['cena_m2'].iloc[-1]
            if 'cena_lag2' in X.columns:
                cechy_prognozy['cena_lag2'] = dane_kompletne['cena_m2'].iloc[-2] if len(dane_kompletne) > 1 else dane_kompletne['cena_m2'].iloc[-1]
            if 'cena_ma4' in X.columns:
                cechy_prognozy['cena_ma4'] = dane_kompletne['cena_m2'].tail(4).mean()
        else:
            # Użyj poprzednich prognoz jako lag
            if 'cena_lag1' in X.columns:
                cechy_prognozy['cena_lag1'] = prognozy[-1] if i > 0 else dane_kompletne['cena_m2'].iloc[-1]
            if 'cena_lag2' in X.columns:
                cechy_prognozy['cena_lag2'] = prognozy[-2] if i > 1 else (prognozy[-1] if i > 0 else dane_kompletne['cena_m2'].iloc[-1])
            if 'cena_ma4' in X.columns:
                if i >= 3:
                    cechy_prognozy['cena_ma4'] = np.mean(prognozy[-4:])
                else:
                    cechy_prognozy['cena_ma4'] = dane_kompletne['cena_m2'].tail(4).mean()
        
        # Przygotuj dane wejściowe dla modelu
        X_prognoza = pd.DataFrame([cechy_prognozy])[X.columns]
        
        # Wykonaj prognozę
        if najlepsza_nazwa == 'Linear Regression':
            X_prognoza_scaled = scaler.transform(X_prognoza)
            prognoza = model.predict(X_prognoza_scaled)[0]
        else:
            prognoza = model.predict(X_prognoza)[0]
        
        prognozy.append(prognoza)
        
        # Utwórz datę prognozy
        data_prognozy = pd.to_datetime(f"{nowy_rok}-{(nowy_kwartal-1)*3+1:02d}-01")
        daty_prognoz.append(data_prognozy)
        
        print(f"Q{nowy_kwartal} {nowy_rok}: {prognoza:,.0f} PLN/m²")
    
    return prognozy, daty_prognoz, najlepsza_nazwa

def wizualizacja_prognoz(df, prognozy, daty_prognoz, nazwa_modelu):
    """
    Tworzy wizualizację z prognozami
    """
    print("📊 Tworzenie wizualizacji prognoz...")
    
    fig = go.Figure()
    
    # Dane historyczne dla Warszawy - rynek pierwotny
    dane_hist = df[(df['miasto'] == 'Warszawa') & (df['rynek'] == 'pierwotny')].sort_values('data')
    
    # Dodaj dane historyczne
    fig.add_trace(go.Scatter(
        x=dane_hist['data'],
        y=dane_hist['cena_m2'],
        mode='lines+markers',
        name='Dane historyczne',
        line=dict(color='blue', width=2),
        marker=dict(size=6)
    ))
    
    # Dodaj prognozy
    fig.add_trace(go.Scatter(
        x=daty_prognoz,
        y=prognozy,
        mode='lines+markers',
        name=f'Prognozy ({nazwa_modelu})',
        line=dict(color='red', width=2, dash='dash'),
        marker=dict(size=8, symbol='diamond')
    ))
    
    # Połącz ostatni punkt historyczny z pierwszą prognozą
    if len(dane_hist) > 0:
        fig.add_trace(go.Scatter(
            x=[dane_hist['data'].iloc[-1], daty_prognoz[0]],
            y=[dane_hist['cena_m2'].iloc[-1], prognozy[0]], 
            mode='lines',
            line=dict(color='red', width=2, dash='dash'),
            showlegend=False
        ))
    
    # Dodaj przedziały ufności (uproszczone)
    margines_bledu = np.array(prognozy) * 0.1  # ±10% jako przybliżony przedział ufności
    
    fig.add_trace(go.Scatter(
        x=daty_prognoz + daty_prognoz[::-1],
        y=(np.array(prognozy) + margines_bledu).tolist() + (np.array(prognozy) - margines_bledu)[::-1].tolist(),
        fill='toself',
        fillcolor='rgba(255,0,0,0.2)',
        line=dict(color='rgba(255,255,255,0)'),
        name='Przedział ufności ±10%',
        showlegend=True
    ))
    
    # Aktualizacja layoutu
    fig.update_layout(
        title='Prognozy Cen Mieszkań - Warszawa (Rynek Pierwotny)',
        xaxis_title='Rok',
        yaxis_title='Cena (PLN/m²)',
        height=600,
        hovermode='x unified'
    )
    
    # Dodaj linię pionową oddzielającą dane historyczne od prognoz
    if len(dane_hist) > 0:
        fig.add_vline(
            x=dane_hist['data'].iloc[-1],
            line_dash='dot',
            line_color='gray',
            annotation_text='Początek prognoz'
        )
    
    return fig

# Tworzenie prognoz
if 'wyniki_modeli' in locals():
    prognozy, daty_prognoz, najlepszy_model = utworz_prognozy_cenowe(
        wyniki_modeli, scaler, dane_kompletne, X, y
    )
    
    # Wizualizacja prognoz
    wykres_prognoz = wizualizacja_prognoz(df_mieszkania, prognozy, daty_prognoz, najlepszy_model)
    wykres_prognoz.show()
    
    # Oblicz statystyki prognoz
    sredni_wzrost_kwartalny = np.mean(np.diff(prognozy) / prognozy[:-1]) * 100
    roczny_wzrost = ((prognozy[3] / prognozy[0]) ** (1/1) - 1) * 100
    
    print(f"\n📈 ANALIZA PROGNOZ:")
    print(f"Średni wzrost kwartalny: {sredni_wzrost_kwartalny:.2f}%")
    print(f"Przewidywany wzrost roczny: {roczny_wzrost:.2f}%")
    print(f"Cena początkowa: {prognozy[0]:,.0f} PLN/m²")
    print(f"Cena końcowa (2 lata): {prognozy[-1]:,.0f} PLN/m²")
    print(f"Całkowity wzrost (2 lata): {((prognozy[-1]/prognozy[0])-1)*100:.1f}%")


In [ ]:
# Komórka 9: Podsumowanie projektu i wnioski
def wygeneruj_raport_koncowy(df, korelacje, testy, wyniki_modeli, prognozy):
    """
    Generuje końcowy raport z projektu
    """
    print("📋 KOMPLEKSOWY RAPORT ANALIZY POLSKIEGO RYNKU NIERUCHOMOŚCI")
    print("="*70)
    
    # 1. Podsumowanie danych
    print("\n1. 📊 PODSUMOWANIE DANYCH:")
    print(f"   • Liczba obserwacji: {len(df):,}")
    print(f"   • Zakres czasowy: {df['kwartal'].min()} - {df['kwartal'].max()}")
    print(f"   • Analizowane miasta: {', '.join(df['miasto'].unique())}")
    print(f"   • Analizowane rynki: {', '.join(df['rynek'].unique())}")
    
    # 2. Kluczowe statystyki
    print("\n2. 📈 KLUCZOWE STATYSTYKI CENOWE:")
    stats_mieszkania = df.groupby(['miasto', 'rynek'])['cena_m2'].agg(['mean', 'std', 'min', 'max'])
    for (miasto, rynek), row in stats_mieszkania.iterrows():
        print(f"   • {miasto} ({rynek}): średnia {row['mean']:,.0f} PLN/m² (σ={row['std']:,.0f})")
    
    # 3. Analiza korelacji
    print("\n3. 🔗 ANALIZA KORELACJI:")
    print("   Korelacje cen mieszkań z wskaźnikami ekonomicznymi:")
    for segment, stats in korelacje.items():
        if 'Warszawa' in segment:
            inflacja_opis = "silna" if abs(stats['inflacja_corr']) > 0.5 else "słaba" if abs(stats['inflacja_corr']) > 0.3 else "bardzo słaba"
            wibor_opis = "silna" if abs(stats['wibor_corr']) > 0.5 else "słaba" if abs(stats['wibor_corr']) > 0.3 else "bardzo słaba"
            print(f"   • {segment}: Inflacja r={stats['inflacja_corr']:.3f} ({inflacja_opis}), WIBOR r={stats['wibor_corr']:.3f} ({wibor_opis})")
    
    # 4. Trendy czasowe
    print("\n4. 📈 TRENDY CZASOWE:")
    for segment, stats in testy.items():
        if 'Warszawa' in segment and not np.isnan(stats['trend_slope']):
            trend_opis = "wzrostowy" if stats['trend_slope'] > 0 else "spadkowy"
            istotnosc = "istotny" if stats['trend_p'] < 0.05 else "nieistotny"
            print(f"   • {segment}: {stats['trend_slope']:+.0f} PLN/m²/kw ({trend_opis}, statystycznie {istotnosc})")
    
    # 5. Wydajność modeli
    print("\n5. 🤖 WYDAJNOŚĆ MODELI PREDYKCYJNYCH:")
    najlepszy = max(wyniki_modeli.keys(), key=lambda k: wyniki_modeli[k]['r2'])
    print(f"   Najlepszy model: {najlepszy}")
    for nazwa, wyniki in wyniki_modeli.items():
        ocena = "doskonała" if wyniki['r2'] > 0.9 else "bardzo dobra" if wyniki['r2'] > 0.8 else "dobra" if wyniki['r2'] > 0.7 else "przeciętna"
        print(f"   • {nazwa}: R²={wyniki['r2']:.4f} ({ocena}), błąd średni {wyniki['mae']:,.0f} PLN/m²")
    
    # 6. Prognozy
    print("\n6. 🔮 PROGNOZY CENOWE (2 lata):")
    wzrost_2_lata = ((prognozy[-1]/prognozy[0])-1)*100
    print(f"   • Prognoza ceny początkowej: {prognozy[0]:,.0f} PLN/m²")
    print(f"   • Prognoza ceny końcowej: {prognozy[-1]:,.0f} PLN/m²")
    print(f"   • Oczekiwany wzrost: {wzrost_2_lata:.1f}% w ciągu 2 lat")
    print(f"   • Średni wzrost roczny: {wzrost_2_lata/2:.1f}%")
    
    # 7. Wnioski biznesowe
    print("\n7. 💡 KLUCZOWE WNIOSKI I REKOMENDACJE:")
    print("   WNIOSKI ANALITYCZNE:")
    print("   • Polski rynek nieruchomości wykazuje stabilny trend wzrostowy")
    print("   • Ceny mieszkań są słabo skorelowane z klasycznymi wskaźnikami ekonomicznymi")
    print("   • Warszawa pozostaje najdroższym rynkiem z najwyższą dynamiką wzrostu")
    print("   • Rynek pierwotny systematycznie droższy od wtórnego")
    
    print("\n   REKOMENDACJE INWESTYCYJNE:")
    if wzrost_2_lata > 15:
        print("   • ✅ Prognoza wskazuje na atrakcyjną inwestycję długoterminową")
    elif wzrost_2_lata > 8:
        print("   • ⚖️ Umiarkowana atrakcyjność inwestyjna")
    else:
        print("   • ⚠️ Niska dynamika wzrostu - rozważna ocena ryzyka")
    
    print("   • Diversyfikacja między miastami może redukować ryzyko")
    print("   • Monitoring wskaźników makroekonomicznych kluczowy dla timing'u")
    
    # 8. Ograniczenia analizy
    print("\n8. ⚠️ OGRANICZENIA I ZASTRZEŻENIA:")
    print("   • Analiza oparta na danych agregowanych (brak danych jednostkowych)")
    print("   • Prognozy zakładają stabilność warunków makroekonomicznych")
    print("   • Nie uwzględniono czynników geopolitycznych i pandemicznych")
    print("   • Model może nie uwzględniać wszystkich czynników lokalnych")
    
    return True

# Wywołanie raportu końcowego
if all(x in locals() for x in ['df_mieszkania', 'korelacje', 'testy', 'wyniki_modeli', 'prognozy']):
    wygeneruj_raport_koncowy(df_mieszkania, korelacje, testy, wyniki_modeli, prognozy)
    
    print("\n" + "="*70)
    print("✅ PROJEKT ZAKOŃCZONY POMYŚLNIE!")
    print("📊 Wszystkie analizy wykonane, wizualizacje utworzone, modele wytrenowane")
    print("🎯 Raport dostępny powyżej, wykresy interaktywne wyświetlone")
    print("💾 Zalecane: Zapisz notebook dla przyszłego wykorzystania")
    print("="*70)
else:
    print("❌ Nie wszystkie komponenty projektu są dostępne. Upewnij się, że wykonałeś wszystkie poprzednie komórki.")


In [ ]:
# Komórka 10: Eksport wyników (opcjonalnie)
def eksportuj_wyniki(df, prognozy, daty_prognoz):
    """
    Eksportuje wyniki do plików Excel i CSV
    """
    print("💾 Eksportowanie wyników...")
    
    try:
        # Eksport danych przetworzonych
        df.to_csv('analiza_mieszkania_dane.csv', index=False, encoding='utf-8')
        df.to_excel('analiza_mieszkania_dane.xlsx', index=False)
        
        # Eksport prognoz
        df_prognozy = pd.DataFrame({
            'data': daty_prognoz,
            'prognoza_cena_m2': prognozy,
            'kwartal': [f"Q{d.quarter} {d.year}" for d in daty_prognoz]
        })
        df_prognozy.to_csv('prognozy_cenowe.csv', index=False, encoding='utf-8')
        df_prognozy.to_excel('prognozy_cenowe.xlsx', index=False)
        
        print("✅ Wyniki wyeksportowane do plików:")
        print("   • analiza_mieszkania_dane.csv/.xlsx")
        print("   • prognozy_cenowe.csv/.xlsx")
        
    except Exception as e:
        print(f"❌ Błąd podczas eksportu: {e}")

# Opcjonalny eksport wyników
# eksportuj_wyniki(df_mieszkania, prognozy, daty_prognoz)

print("🎉 PROJEKT KOMPLETNY!")
print("Wszystkie analizy zostały wykonane. Sprawdź wykresy interaktywne powyżej.")
